In [14]:
import open3d as o3d
import numpy as np
import math
from scipy.spatial.transform import Rotation as R



def default_visualization(object, zoom=1.0):
    azimuth_deg = -45
    elevation_deg = -135

    az = math.radians(azimuth_deg)
    el = math.radians(elevation_deg)

    # spherical → cartesian (camera front vector)
    front = np.array([
    math.cos(el) * math.cos(az),
    math.cos(el) * math.sin(az),
    math.sin(el)])

    # Open3D camera looks toward object
    front = -front
    lookat = [0, 0, 0]
    up = [0, 0, 1]  # typical CAD Z-up

    o3d.visualization.draw_geometries(
        object,
        window_name="Default Visualization",
        width=1024,
        height=768,
        lookat=[0, 0, 0],
        up=[0, 0, 1],
        front=front,
        zoom=zoom
    )

SOURCE_PATH = "workpiece/first_object.stl" #  CAD model

visualization = []

mesh = o3d.io.read_triangle_mesh(SOURCE_PATH)
mesh.compute_vertex_normals() # Makes the shading look better
    
target_points_raw = o3d.geometry.PointCloud()
target_points_raw.points = mesh.vertices
target_points_raw.normals = mesh.vertex_normals
target_points_raw.paint_uniform_color([1, 0, 0])
visualization.append(target_points_raw)

pcd_all = o3d.geometry.PointCloud()
pcd_all = mesh.sample_points_poisson_disk(number_of_points=5000)
pcd_all.paint_uniform_color([0.25, 0, 0])
# visualization.append(pcd_all)

world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=50, origin=[0, 0, 0])
visualization.append(world_frame)


default_visualization([mesh]+visualization, zoom=1.0)

In [9]:
# Removes duplicate points and averages the normals

target_points_clean = target_points_raw.voxel_down_sample(voxel_size=0.001) 

print(f"Before removing duplicate: {len(target_points_raw.points)} points")
print(f"After removing duplicate: {len(target_points_clean.points)} points")

default_visualization([target_points_clean, mesh], zoom=1.0)


Before removing duplicate: 3254 points
After removing duplicate: 1092 points


In [10]:
# Reduce points that are too close to each other by using a voxel grid approach

closeness_threshold = 5.0  # Units (e.g., mm)

# This returns a new point cloud where points within 
# the same 2x2x2 cube are combined.
target_points_downsampled = target_points_clean.voxel_down_sample(voxel_size=closeness_threshold)

print(f"Before combining: {len(target_points_clean.points)} points")
print(f"After combining: {len(target_points_downsampled.points)} points")

default_visualization([target_points_downsampled, mesh], zoom=1.0)

Before combining: 1092 points
After combining: 161 points


In [11]:
# Generate 6D viewpoints (X, Y, Z, Roll, Pitch, Yaw) for each point

def get_euler_from_matrix(R):
    """Extracts Roll, Pitch, Yaw from a 3x3 Rotation Matrix"""
    # Using the standard XYZ convention
    pitch = math.asin(-R[2, 0])
    roll = math.atan2(R[2, 1], R[2, 2])
    yaw = math.atan2(R[1, 0], R[0, 0])
    
    # Return as degrees for easier reading
    return [math.degrees(roll), math.degrees(pitch), math.degrees(yaw)]


viewpoints_6d = []
viewpoints_visualization = []
offset_distance = 600.0 # 300 mm from the surface along the normal direction

for i in range(len(target_points_downsampled.points)):
    P = np.asarray(target_points_downsampled.points[i])
    N = np.asarray(target_points_downsampled.normals[i])
    
    # 1. Position the camera at the offset
    cam_pos = P + (N * offset_distance)
    
    # 2. Define the target direction
    # We want the camera's Z-axis to point toward P. 
    # Since the camera is at cam_pos, the direction is (P - cam_pos), 
    # which is exactly the same as -N.
    z_axis = -N / np.linalg.norm(N) # Ensure it's a unit vector
    
    # 3. Find the rotation matrix that aligns the world Z [0,0,1] to our z_axis
    # We use a helper function to create a rotation matrix from a direction
    # Create an arbitrary "up" vector (world Y or X) to build the coordinate system
    up = np.array([0, 1, 0]) if abs(z_axis[1]) < 0.9 else np.array([1, 0, 0])
    x_axis = np.cross(up, z_axis)
    x_axis /= np.linalg.norm(x_axis)
    y_axis = np.cross(z_axis, x_axis)
    
    # Build the 3x3 rotation matrix
    R = np.column_stack((x_axis, y_axis, z_axis))

    # 4. Create and transform the coordinate frame
    cam_frame_visualization = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10)
    
    # Apply rotation first, then translation
    cam_frame_visualization.rotate(R, center=(0, 0, 0)) 
    cam_frame_visualization.translate(cam_pos)
    
    viewpoints_visualization.append(cam_frame_visualization)

    # 1. Get the 3 angles from the matrix
    angles = get_euler_from_matrix(R)
    
    # 2. Combine Position (3D) + Angles (3D) = 6D
    pose_6d = [
        float(cam_pos[0]), float(cam_pos[1]), float(cam_pos[2]), # X, Y, Z
        angles[0],  angles[1],  angles[2]   # Roll, Pitch, Yaw
    ]
    
    viewpoints_6d.append(pose_6d)

print("Generated viewpoints preview (X, Y, Z, Roll, Pitch, Yaw):")
for i in range(5):
    print(viewpoints_6d[i])  # Print the first 5 | x y z roll pitch yaw

default_visualization(viewpoints_visualization + [target_points_downsampled, mesh], zoom=0.6)

Generated viewpoints preview (X, Y, Z, Roll, Pitch, Yaw):
[-342.17665613055414, -168.0, 184.99759086539657, 151.43467678580504, 65.0000007016778, 180.0]
[1.6404655411861312e-13, -164.00000000000003, 413.57978496320493, 152.6366472067176, -2.469094323830403e-14, 180.0]
[64.31897585928452, -168.0, -350.27120896562485, -28.096037902779326, -9.999995471460204, -0.0]
[17.15556275551733, -163.99999999999991, 14.500006018927063, -175.07256588513664, -89.9999982924527, 90.0]
[222.1588896839868, -164.00000000000003, -258.0760813481904, -27.982560992212946, -39.08301677065767, -0.0]


In [170]:
# Optional Reduction Viewpoint

z_threshold = 300.0 # mm

# Option 1: Using zip to create both new lists at once
viewpoints_6d_reduced = []
viewpoints_visualization_reduced = []
target_points_reduced = []
reduced_index = []
i = 0

for vp, vis in zip(viewpoints_6d, viewpoints_visualization):
    if vp[2] >= z_threshold:
        viewpoints_6d_reduced.append(vp)
        viewpoints_visualization_reduced.append(vis)
        reduced_index.append(i)
    i += 1

target_points_reduced = target_points_downsampled.select_by_index(reduced_index)

print("Before reduction: ", len(viewpoints_6d))
print("After reduction: ", len(viewpoints_6d_reduced))

default_visualization(viewpoints_visualization_reduced + [target_points_reduced, mesh], zoom=1.25)


Before reduction:  161
After reduction:  27


In [175]:
from scipy.spatial.transform import Rotation as R

# Check the visualization of one viewpoint in detail, including the camera position, target point, and the line connecting them.

def pose_to_matrix(pose):
    """
    Converts [x, y, z, roll, pitch, yaw] in degrees to a 4x4 transformation matrix.
    If zyz=True, assumes the input is in ZYZ Euler Angles, otherwise XYZ Euler Angles.
    
    Args:
    - pose (list or array): [x, y, z, roll, pitch, yaw] in degrees.
    - zyz (bool): If True, interprets the last three values as ZYZ Euler angles. 
                  If False, uses XYZ Euler angles (default).
    
    Returns:
    - T (ndarray): The 4x4 homogeneous transformation matrix.
    """
    x, y, z = pose[:3]   # Translation vector
    roll, pitch, yaw = pose[3:]  # Rotation angles in degrees
    
    # Convert XYZ Euler Angles to a 3x3 rotation matrix
    rot_matrix = R.from_euler('xyz', [roll, pitch, yaw], degrees=True).as_matrix()

    # Build the 4x4 Homogeneous Transformation Matrix
    T = np.eye(4)  # Start with the identity matrix (4x4)
    T[:3, :3] = rot_matrix  # Set the upper-left 3x3 part to the rotation matrix
    T[:3, 3] = [x, y, z]    # Set the upper-right 3x1 part to the translation vector
    return T

def inspect_viewpoint(target_idx, target_points_downsampled, viewpoints_6d, viewpoints_visualization, mesh=None):
    target_point_coords = np.asarray(target_points_downsampled.points[target_idx])

    sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.25)
    sphere.translate(target_point_coords)
    sphere.paint_uniform_color([1, 0, 0])

    selected_camera = viewpoints_visualization[target_idx]

    cam_pos = np.array(viewpoints_6d[target_idx][:3])

    line_points = [target_point_coords, cam_pos]
    line_indices = [[0, 1]]

    connector_line = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(line_points),
        lines=o3d.utility.Vector2iVector(line_indices)
    )
    connector_line.paint_uniform_color([0, 1, 0])

    print(f"\n--- Inspecting Viewpoint Index: {target_idx} ---")
    print(f"Target Surface Point: {target_point_coords}")
    print(f"Camera Position (6D): {viewpoints_6d[target_idx]}")

    return sphere, selected_camera, connector_line, cam_pos, target_point_coords

def camera_pov_visualization(geoms, cam_pos, target_point, show_world_frame=False, show_camera_frame=False, save=0, index=0):
    
    vis = o3d.visualization.Visualizer()
    if save:
        vis.create_window(visible=False)
    else:
        vis.create_window()

    # world frame
    if show_world_frame:
        world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
            size=50
        )
        vis.add_geometry(world_frame)

    for g in geoms:
        vis.add_geometry(g)

    ctr = vis.get_view_control()

    # STEP 1 — camera + target
    cam_pos = np.asarray(cam_pos, dtype=float)
    target_point = np.asarray(target_point, dtype=float)
    # print("cam_pos:", cam_pos)
    # print("target_point:", target_point)

    # STEP 2 — forward (camera → target)  
    forward = target_point - cam_pos
    forward = forward / np.linalg.norm(forward)
    # print("forward:", forward)
    
    # STEP 3 — stable world up
    world_up = np.array([0.0, 0.0, 1.0])
    if abs(np.dot(forward, world_up)) > 0.99:
        world_up = np.array([0.0, 1.0, 0.0])
    # print("world_up:", world_up)
    
    # STEP 4 — right vector
    right = np.cross(forward, world_up)
    right = right / np.linalg.norm(right)
    # print("right:", right)
    
    # STEP 5 — true up
    up = np.cross(right, forward)
    up = up / np.linalg.norm(up)
    # print("up:", up)
    
    # OPTIONAL — visualize camera frame
    if show_camera_frame:
        cam_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
            size=50)
        cam_frame.rotate(np.stack([right, up, forward], axis=1), center=(0, 0, 0))
        cam_frame.translate(cam_pos)
        vis.add_geometry(cam_frame)

    
    # STEP 6 — build rotation (your verified correct version)
    R = np.stack([right, -up, forward], axis=1)
    # print("R:\n", R)
    # print("det(R):", np.linalg.det(R))

    # STEP 7 — extrinsic
    extrinsic = np.eye(4)
    extrinsic[:3, :3] = R.T
    extrinsic[:3, 3] = -R.T @ cam_pos
    # print("extrinsic:\n", extrinsic)

    # STEP 8 — apply camera
    param = ctr.convert_to_pinhole_camera_parameters()
    param.extrinsic = extrinsic
    ctr.convert_from_pinhole_camera_parameters(param)
    
    vis.poll_events()
    vis.update_renderer()

    if save == 1:
        vis.capture_screen_image(f"viewpoints_candidate/viewpoint_object_{index}.png")
    elif save ==2:
        vis.capture_screen_image(f"viewpoints_candidate/viewpoint_pcd_{index}.png")
    else:
        vis.run()
    vis.destroy_window()

check_all = True

if not check_all:
    index = 2
    sphere, selected_camera, connector_line, cam_pos, target_point_coords = inspect_viewpoint(
        index, target_points_reduced, viewpoints_6d_reduced, viewpoints_visualization_reduced, mesh)

    default_visualization([mesh, selected_camera, sphere, connector_line, world_frame], zoom=1.25)
    camera_pov_visualization([mesh, sphere, connector_line], cam_pos, target_point_coords, show_world_frame=False, show_camera_frame=False)

    np.set_printoptions(precision=6, suppress=True, formatter={'all': lambda x: f'{x:0.6f}'}) # normal
    # print(pose_to_matrix(viewpoints_6d_reduced[index]))
    formatted_matrix = np.array2string(pose_to_matrix(viewpoints_6d_reduced[index]), separator=', ', precision=6, suppress_small=True)
    print(formatted_matrix)

else:
    for index in range(len(viewpoints_6d_reduced)):
        sphere, selected_camera, connector_line, cam_pos, target_point_coords = inspect_viewpoint(
        index, target_points_reduced, viewpoints_6d_reduced, viewpoints_visualization_reduced, mesh)

        default_visualization([mesh, selected_camera, sphere, connector_line, world_frame], zoom=0.75)


--- Inspecting Viewpoint Index: 0 ---
Target Surface Point: [-0.000000 36.000000 27.135899]
Camera Position (6D): [1.6404655411861312e-13, -164.00000000000003, 413.57978496320493, 152.6366472067176, -2.469094323830403e-14, 180.0]

--- Inspecting Viewpoint Index: 1 ---
Target Surface Point: [-5.679261 40.000000 21.097313]
Camera Position (6D): [-136.1095751725445, 240.0, 323.4687840967083, -148.72788662565034, 23.33333226808214, 180.0]

--- Inspecting Viewpoint Index: 2 ---
Target Surface Point: [10.194810 32.000000 11.156905]
Camera Position (6D): [258.5961647642761, -168.0, 307.19013062557826, 152.63664931749554, -39.999998206328954, 180.0]

--- Inspecting Viewpoint Index: 3 ---
Target Surface Point: [13.384773 32.000000 27.102028]
Camera Position (6D): [-20.687126486685493, -168.0, 416.5452160887139, 152.9057377025625, 5.00000514687876, 180.0]

--- Inspecting Viewpoint Index: 4 ---
Target Surface Point: [5.381171 40.000000 21.062131]
Camera Position (6D): [183.18339464298253, 240.0,

In [176]:
# Remove hidden points from the camera's perspective using Open3D's Hidden Point Removal (HPR) algorithm

def hidden_point_removal_from_camera(pcd, cam_pos, radius_scale=100):
    """
    Returns a visible-point point cloud using Open3D HPR.
    """

    pts = np.asarray(pcd.points)
    diameter = np.linalg.norm(pcd.get_max_bound() - pcd.get_min_bound())

    radius = diameter * radius_scale

    _, pt_map = pcd.hidden_point_removal(cam_pos, radius)
    visible_pcd = pcd.select_by_index(pt_map)

    visible_pcd.paint_uniform_color([0, 1, 0])  # Visible points in green

    return visible_pcd

# Heuristic Approach
radius_scale = 5
center = pcd_all.get_center()
d_cam = np.linalg.norm(cam_pos - center)
radius = d_cam * radius_scale

visible_pcd = hidden_point_removal_from_camera( pcd_all, cam_pos, radius)

camera_pov_visualization([visible_pcd, sphere, connector_line], cam_pos, target_point_coords, show_world_frame=False, show_camera_frame=False)


In [177]:
# Generate viewpoints for all points and save them to a file

print("Final feasible viewpoints count:", len(viewpoints_6d_reduced))

for i in range(len(viewpoints_6d_reduced)):

    # Save simulated Image from Viewpoint
    sphere, _, _, cam_pos, target_point_coords = inspect_viewpoint(
        i, target_points_downsampled, viewpoints_6d_reduced, viewpoints_visualization_reduced, mesh)
    camera_pov_visualization([mesh, sphere], cam_pos, target_point_coords, save=1, index=i)

    # Save simulated PCD from Viewpoint
    radius_scale = 5
    center = pcd_all.get_center()
    d_cam = np.linalg.norm(cam_pos - center)
    radius = d_cam * radius_scale

    visible_pcd = hidden_point_removal_from_camera(pcd_all, viewpoints_6d_reduced[i][:3], radius)
    camera_pov_visualization([visible_pcd, sphere], cam_pos, target_point_coords, save=2, index=i)
    o3d.io.write_point_cloud(f"viewpoints_candidate/viewpoint_simulated_{i}.pcd", visible_pcd)

    # Save the 6D pose as a numpy array (X, Y, Z, Roll, Pitch, Yaw)
    pose_6d = pose_to_matrix(viewpoints_6d_reduced[i])
    np.save(f"viewpoints_candidate/viewpoint_pose_{i}.npy", pose_6d)

output = np.load(f"viewpoints_candidate/viewpoint_pose_0.npy")
print("\nExample of output file transformation:\n", output)

Final feasible viewpoints count: 27

--- Inspecting Viewpoint Index: 0 ---
Target Surface Point: [-9.240157 32.000000 29.746757]
Camera Position (6D): [1.6404655411861312e-13, -164.00000000000003, 413.57978496320493, 152.6366472067176, -2.469094323830403e-14, 180.0]

--- Inspecting Viewpoint Index: 1 ---
Target Surface Point: [-0.000000 36.000000 27.135899]
Camera Position (6D): [-136.1095751725445, 240.0, 323.4687840967083, -148.72788662565034, 23.33333226808214, 180.0]

--- Inspecting Viewpoint Index: 2 ---
Target Surface Point: [-0.734649 32.000000 18.666403]
Camera Position (6D): [258.5961647642761, -168.0, 307.19013062557826, 152.63664931749554, -39.999998206328954, 180.0]

--- Inspecting Viewpoint Index: 3 ---
Target Surface Point: [-0.086957 36.000000 14.500000]
Camera Position (6D): [-20.687126486685493, -168.0, 416.5452160887139, 152.9057377025625, 5.00000514687876, 180.0]

--- Inspecting Viewpoint Index: 4 ---
Target Surface Point: [-15.154535 36.000000 34.114938]
Camera Posi